# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
# Import API key
from api_keys import g_key

In [10]:
#load csv file
file_path ='output_data\cities.csv'
cities_df=pd.read_csv(file_path)
cities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


In [13]:
# Configure gmaps.
#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.

fig = gmaps.figure()
lat_lng=cities_df[['Lat','Lng']]
humidity=cities_df['Humidity']

heat_layer=gmaps.heatmap_layer(lat_lng, 
                               weights=humidity, 
                               dissipating=False, 
                               max_intensity=10, 
                               point_radius=1)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [15]:
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values
cities_df.info()
city_weather_df=pd.DataFrame(cities_df, columns = ['City','Country','Lat','Lng','Cloudiness','Humidity','Max Temp','Wind Speed'])

city_weather_df=city_weather_df.dropna()

city_weather_df=city_weather_df.loc[(city_weather_df['Wind Speed']<10)
                                    &(city_weather_df["Cloudiness"]==0)
                                    &(city_weather_df["Max Temp"]>= 70)
                                    & (city_weather_df["Max Temp"]<=80)]
city_weather_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     548 non-null    int64  
 1   City        548 non-null    object 
 2   Cloudiness  548 non-null    int64  
 3   Country     544 non-null    object 
 4   Date        548 non-null    int64  
 5   Humidity    548 non-null    int64  
 6   Lat         548 non-null    float64
 7   Lng         548 non-null    float64
 8   Max Temp    548 non-null    float64
 9   Wind Speed  548 non-null    float64
dtypes: float64(4), int64(4), object(2)
memory usage: 42.9+ KB


,City,Country,Lat,Lng,Cloudiness,Humidity,Max Temp,Wind Speed
70,ponta do sol,BR,-20.63,-46.00,0,57,73.68,5.32
88,cidreira,BR,-30.17,-50.22,0,66,76.56,5.50
138,morondava,MG,-20.30,44.28,0,69,79.08,9.06
139,vaini,IN,15.34,74.49,0,88,78.90,1.59
176,nador,MA,35.17,-2.93,0,40,75.20,6.93
297,mogok,MM,22.92,96.51,0,51,72.24,2.77
444,birjand,IR,32.86,59.22,0,26,71.60,9.17


In [16]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame

hotel_df=city_weather_df
hotel_df['Hotel Name']=''
hotel_df

,City,Country,Lat,Lng,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,0,57,73.68,5.32,
88,cidreira,BR,-30.17,-50.22,0,66,76.56,5.50,
138,morondava,MG,-20.30,44.28,0,69,79.08,9.06,
139,vaini,IN,15.34,74.49,0,88,78.90,1.59,
176,nador,MA,35.17,-2.93,0,40,75.20,6.93,
297,mogok,MM,22.92,96.51,0,51,72.24,2.77,
444,birjand,IR,32.86,59.22,0,26,71.60,9.17,


In [17]:
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [18]:
#iterate throught API request to fill list

for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    city=row['City']
    
    params['location']=f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("No results found")
        print("-------------------------")

#delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 70: ponta do sol.
Closest hotel in ponta do sol is Escarpas Resort
--------------------Ending Search
Retrieving Results for Index 88: cidreira.
Closest hotel in cidreira is Hotel Castelo
--------------------Ending Search
Retrieving Results for Index 138: morondava.
Closest hotel in morondava is Trecicogne
--------------------Ending Search
Retrieving Results for Index 139: vaini.
Closest hotel in vaini is Dandeli Dreams Jungle Resort
--------------------Ending Search
Retrieving Results for Index 176: nador.
Closest hotel in nador is Hôtel Mercure Rif Nador
--------------------Ending Search
Retrieving Results for Index 297: mogok.
Closest hotel in mogok is MOGOK HILL HOTEL
--------------------Ending Search
Retrieving Results for Index 444: birjand.
Closest hotel in birjand is هتل پرويز
--------------------Ending Search


In [19]:
hotel_df

,City,Country,Lat,Lng,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,0,57,73.68,5.32,Escarpas Resort
88,cidreira,BR,-30.17,-50.22,0,66,76.56,5.50,Hotel Castelo
138,morondava,MG,-20.30,44.28,0,69,79.08,9.06,Trecicogne
139,vaini,IN,15.34,74.49,0,88,78.90,1.59,Dandeli Dreams Jungle Resort
176,nador,MA,35.17,-2.93,0,40,75.20,6.93,Hôtel Mercure Rif Nador
297,mogok,MM,22.92,96.51,0,51,72.24,2.77,MOGOK HILL HOTEL
444,birjand,IR,32.86,59.22,0,26,71.60,9.17,هتل پرويز


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_df

,City,Country,Lat,Lng,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,0,57,73.68,5.32,Escarpas Resort
88,cidreira,BR,-30.17,-50.22,0,66,76.56,5.50,Hotel Castelo
138,morondava,MG,-20.30,44.28,0,69,79.08,9.06,Trecicogne
139,vaini,IN,15.34,74.49,0,88,78.90,1.59,Dandeli Dreams Jungle Resort
176,nador,MA,35.17,-2.93,0,40,75.20,6.93,Hôtel Mercure Rif Nador
297,mogok,MM,22.92,96.51,0,51,72.24,2.77,MOGOK HILL HOTEL
444,birjand,IR,32.86,59.22,0,26,71.60,9.17,هتل پرويز


In [22]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>